## Imports

In [1]:
# pip install --upgrade scikit-learn

In [131]:
import time
from IPython.display import clear_output
import numpy    as np
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt
import sklearn  as skl

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config

set_config(display='diagram') # Useful for display the pipeline

print("Pandas  ", pd.__version__)
print("Sklearn ", skl.__version__) # Try to use 0.24

Pandas   1.3.1
Sklearn  0.24.2


## Get the dataset
- **CLOUD = True**: Download dataset from Kaggle. Necesary for cloud enviroments like COLAB. **Specify your [kaggle credentials](https://www.kaggle.com/docs/api)**.
- **CLOUD = False**: Get the dataset from your local machine. **Specify the data path**.

In [132]:
DATA_PATH = "C:/Users/User/Documents/Strive_AI_Jun_21/M4-Feature Engineering/02 - Robust ML/"

## Load data

In [133]:
df      = pd.read_csv(DATA_PATH + "train.csv", index_col='PassengerId')
df_test = pd.read_csv(DATA_PATH + "test.csv",  index_col='PassengerId')

print("Train DataFrame:", df.shape)
print("Test DataFrame: ", df_test.shape)

Train DataFrame: (891, 11)
Test DataFrame:  (418, 10)


## Check missings

In [134]:
df.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [135]:
df_test.isnull().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

# Exercise 1 (2pts):
Extract the title (Mr, Mrs, ... ) from the "Name" column.

Tips:
- split(',')[1] to get the 2nd part, and remove the surnamename
- split('.')[0] to get the 1str part, and remove the name

In [136]:
# CODE HERE get_Title_from_Name funtion
# Create this function using lambda (not def)
get_Title_from_Name = lambda name: name.split(',')[1].split('.')[0].strip()
df['Title']=df['Name'].apply(lambda x:x.split(',')[1].split('.')[0].strip())
df_test['Title']=df_test['Name'].apply(lambda x:x.split(',')[1].split('.')[0].strip())
print(df['Title'])

PassengerId
1        Mr
2       Mrs
3      Miss
4       Mrs
5        Mr
       ... 
887     Rev
888    Miss
889    Miss
890      Mr
891      Mr
Name: Title, Length: 891, dtype: object


# Exercise 2 (1pts):
Apply the title_dictionary to get a better information about the title. You have to overwrite the Title variable.

In [137]:
title_dictionary = {
    "Capt": "Officer",
    "Col": "Officer",
    "Major": "Officer",
    "Jonkheer": "Royalty",
    "Don": "Royalty",
    "Sir" : "Royalty",
    "Dr": "Officer",
    "Rev": "Officer",
    "the Countess":"Royalty",
    "Mme": "Mrs",
    "Mlle": "Miss",
    "Ms": "Mrs",
    "Mr" : "Mr",
    "Mrs" : "Mrs",
    "Miss" : "Miss",
    "Master" : "Master",
    "Lady" : "Royalty"
}

In [138]:
# Use map to apply the prevous dict
df['Title'] = df['Title'].map(title_dictionary)
df_test['Title'] = df_test['Title'].map(title_dictionary)
print(df['Title'])

PassengerId
1           Mr
2          Mrs
3         Miss
4          Mrs
5           Mr
        ...   
887    Officer
888       Miss
889       Miss
890         Mr
891         Mr
Name: Title, Length: 891, dtype: object


# Exercise OPTINAL (0pts):
Try to extract some information from the feature **Ticket**. Search on Internet if that colum has some kind of information.

In [142]:
df_test['Ticket']

PassengerId
892                 330911
893                 363272
894                 240276
895                 315154
896                3101298
               ...        
1305             A.5. 3236
1306              PC 17758
1307    SOTON/O.Q. 3101262
1308                359309
1309                  2668
Name: Ticket, Length: 418, dtype: object

# Exercise OPTIONAL (0pts):
Try to extract some information from the feature **Cabin**. Search on Internet if that colum has some kind of information.

In [143]:
df_test['Cabin']

PassengerId
892      NaN
893      NaN
894      NaN
895      NaN
896      NaN
        ... 
1305     NaN
1306    C105
1307     NaN
1308     NaN
1309     NaN
Name: Cabin, Length: 418, dtype: object

# Preprocessing
For X data, notice that...
- We drop Survived because is the target variable
- We drop Name because we have extracted the Title: Mr, Mrs, ...
- We drop Ticket because it has no information -> see df.Ticket.nunique()
- We drop Cabin because it has a lot of missings (77% are missings)

Then, we identify **numerical** variables and **categorical** variables,

In [146]:
df_new=df.drop(['Survived','Ticket','Cabin','Name'],axis=1)
X =df_new
y=df.Survived
print(X)

             Pclass     Sex   Age  SibSp  Parch     Fare Embarked    Title
PassengerId                                                               
1                 3    male  22.0      1      0   7.2500        S       Mr
2                 1  female  38.0      1      0  71.2833        C      Mrs
3                 3  female  26.0      0      0   7.9250        S     Miss
4                 1  female  35.0      1      0  53.1000        S      Mrs
5                 3    male  35.0      0      0   8.0500        S       Mr
...             ...     ...   ...    ...    ...      ...      ...      ...
887               2    male  27.0      0      0  13.0000        S  Officer
888               1  female  19.0      0      0  30.0000        S     Miss
889               3  female   NaN      1      2  23.4500        S     Miss
890               1    male  26.0      0      0  30.0000        C       Mr
891               3    male  32.0      0      0   7.7500        Q       Mr

[891 rows x 8 columns]


In [147]:
cat_vars=X.select_dtypes(include=[object]).columns.values.tolist()
num_vars =X.select_dtypes(exclude=[object]).columns.values.tolist()
num_vars

['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

In [148]:
# cat_vars  = ['Sex', 'Embarked', 'Title']         # x.select_dtypes(include=[object]).columns.values.tolist()
# num_vars  = ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age'] # x.select_dtypes(exclude=[object]).columns.values.tolist()

print("\nNumerical features:\n", num_vars)
print("\nCategorical features:\n", cat_vars)


Numerical features:
 ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

Categorical features:
 ['Sex', 'Embarked', 'Title']


# Exercise 3 (2pts):
Create a **ColumnTransformer for Tree Models**. You need to create 2 pipelines (one for numerical and other for categories). Remember:
- Categorical pipeline: Some SimpleImputer -> Some Encoder
- Numerical pipeline: Some SimpleImputer -> NO Encoder

In [149]:
"""
num_preprocessing = pipeline.Pipeline(steps=[
  # Some Imputer here
])

cat_preporcessing = pipeline.Pipeline(steps=[
  # Some Imputer here
  # Some Encoder here. Remember to handle_unknown
])

tree_prepro = compose.ColumnTransformer(transformers=[
    numerical,
    categorica
],) # Drop other vars not specified in num_vars or cat_vars

tree_prepro
"""
# A Pipeline is simply a method of chaining ...
# multiple steps together in which the output ...
# of the previous step is used as the input for the next step.

num_preprocessing =pipeline.Pipeline([("imputer",impute.SimpleImputer(strategy="mean"))]) #,("scaler",preprocessing.StandardScaler())

cat_preporcessing =pipeline.Pipeline([("imputer",impute.SimpleImputer(strategy='constant', fill_value='missing')),
                                      ("ordinal",preprocessing.OrdinalEncoder(handle_unknown="use_encoded_value",unknown_value = -1))])
tree_prepro = compose.ColumnTransformer(transformers=[('num_data',num_preprocessing,num_vars),('cat_data',cat_preporcessing,cat_vars),],remainder='drop')
tree_prepro

ColumnTransformer(transformers=[('num_data',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']),
                                ('cat_data',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('ordinal',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1))]),
                                 ['Sex', 'Embarked', 'Title'])])

# Exercise 4 (1pts):
1. Complete the diccionary with some Tree Models.
2. Then we put each model in a Pipeline where:
   - first is the prepocessing with the column Transformer
   - Then is the Tree model
3. Display the fullpipeline of the LGBMClassifier

In [150]:
from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier

In [151]:
"""
tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees":  ExtraTreesClassifier()
  "Random Forest": RandomForestClassifier()
  "AdaBoost":AdaBoostClassifier()
  "Skl GBM":GradientBoostingClassifier()
  "Skl HistGBM":HistGradientBoostingClassifier()
  "XGBoost": XGBClassifier()
  "LightGBM":LGBMClassifier()
  "CatBoost":CatBoostClassifier()

tree_classifiers = add Pipeline dictionary with prepro
"""
tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees":  ExtraTreesClassifier(),
  "Random Forest": RandomForestClassifier(),
  "AdaBoost":AdaBoostClassifier(),
  "Skl GBM":GradientBoostingClassifier(),
  "Skl HistGBM":HistGradientBoostingClassifier(),
  "XGBoost": XGBClassifier(),
  "LightGBM":LGBMClassifier(),
  "CatBoost":CatBoostClassifier()
}
trf =DecisionTreeClassifier()
trf2=XGBClassifier()
from sklearn import set_config
set_config(display='diagram')

for tree, method in tree_classifiers.items():
    tree_classifiers[tree]=pipeline.Pipeline([("preprocessing", tree_prepro),(tree,method)])

# tree_classifiers["LightGBM"]
tree_classifiers['LightGBM']

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_data',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['Pclass', 'Age', 'SibSp',
                                                   'Parch', 'Fare']),
                                                 ('cat_data',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('ordinal',
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  unknown_value=-1))]),
                                                  ['Sex', 'Embarked',
                                                   'Title'])])),
                ('LightGBM', LGBMClassifier())])

# Exercise 5 (3pts):
Define a simple split validation strategy with:
- 80% for train
- 20% for validation
- With stratification
- random_state=0

And train all the models in a for loop

In [152]:
"""
x_train, x_val, y_train, y_val = model_selection.train_test_split(
    # CODE HERE
)

Stratify
"""

X_train,X_test,y_train,y_test =model_selection.train_test_split(X,y,test_size=0.2,stratify = y,random_state=0)

results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})


for model_name, model in tree_classifiers.items():
    start_time = time.time()
    
    # FOR EVERY PIPELINE (PREPRO + MODEL) -> TRAIN WITH TRAIN DATA (x_train)
    model.fit(X_train,y_train)
    # GET PREDICTIONS USING x_val
    pred = model.predict(X_test)# CODE HERE

    total_time = time.time() - start_time
    
    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_test, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_test, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              


### BEGIN SOLUTION

results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

C:\Users\User\anaconda3\envs\AI_MachineLearning\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:21:12] WARNING: ..\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Learning rate set to 0.008911
0:	learn: 0.6896767	total: 955us	remaining: 955ms
1:	learn: 0.6833883	total: 2.84ms	remaining: 1.42s
2:	learn: 0.6768278	total: 6.4ms	remaining: 2.13s
3:	learn: 0.6705779	total: 8.61ms	remaining: 2.14s
4:	learn: 0.6645342	total: 10.2ms	remaining: 2.03s
5:	learn: 0.6585854	total: 12ms	remaining: 1.98s
6:	learn: 0.6528304	total: 14.1ms	remaining: 2.01s
7:	learn: 0.6484285	total: 15.7ms	remaining: 1.95s
8:	learn: 0.6427515	total: 17.6ms	remaining: 1.94s
9:	learn: 0.6390466	total: 20.8ms	remaining: 2.06s
10:	learn: 0.6346908	total: 23ms	remaining: 2.06s
11:	learn: 0.6294328	total: 25.3ms	remaining: 2.08s
12:	learn: 0.6244579	total: 26.7ms	remaining: 2.03s
13:	learn: 0.6217645	total: 27.6ms	remaining: 1.94s
14:	le

197:	learn: 0.3711159	total: 318ms	remaining: 1.29s
198:	learn: 0.3705023	total: 320ms	remaining: 1.29s
199:	learn: 0.3701751	total: 322ms	remaining: 1.29s
200:	learn: 0.3699580	total: 323ms	remaining: 1.28s
201:	learn: 0.3699064	total: 324ms	remaining: 1.28s
202:	learn: 0.3694793	total: 325ms	remaining: 1.28s
203:	learn: 0.3693603	total: 327ms	remaining: 1.27s
204:	learn: 0.3690734	total: 328ms	remaining: 1.27s
205:	learn: 0.3689637	total: 329ms	remaining: 1.27s
206:	learn: 0.3686207	total: 332ms	remaining: 1.27s
207:	learn: 0.3680013	total: 333ms	remaining: 1.27s
208:	learn: 0.3676215	total: 335ms	remaining: 1.27s
209:	learn: 0.3670945	total: 337ms	remaining: 1.26s
210:	learn: 0.3666537	total: 338ms	remaining: 1.26s
211:	learn: 0.3663726	total: 340ms	remaining: 1.26s
212:	learn: 0.3660749	total: 342ms	remaining: 1.26s
213:	learn: 0.3656956	total: 347ms	remaining: 1.27s
214:	learn: 0.3652762	total: 349ms	remaining: 1.27s
215:	learn: 0.3651731	total: 351ms	remaining: 1.27s
216:	learn: 

419:	learn: 0.3290896	total: 656ms	remaining: 906ms
420:	learn: 0.3289074	total: 658ms	remaining: 905ms
421:	learn: 0.3286591	total: 659ms	remaining: 903ms
422:	learn: 0.3286204	total: 661ms	remaining: 902ms
423:	learn: 0.3285830	total: 662ms	remaining: 900ms
424:	learn: 0.3283528	total: 664ms	remaining: 898ms
425:	learn: 0.3282800	total: 665ms	remaining: 896ms
426:	learn: 0.3281305	total: 667ms	remaining: 894ms
427:	learn: 0.3280548	total: 669ms	remaining: 894ms
428:	learn: 0.3279357	total: 671ms	remaining: 892ms
429:	learn: 0.3278441	total: 672ms	remaining: 891ms
430:	learn: 0.3276204	total: 674ms	remaining: 890ms
431:	learn: 0.3275066	total: 678ms	remaining: 891ms
432:	learn: 0.3272768	total: 680ms	remaining: 890ms
433:	learn: 0.3272009	total: 681ms	remaining: 889ms
434:	learn: 0.3271131	total: 684ms	remaining: 888ms
435:	learn: 0.3269352	total: 686ms	remaining: 887ms
436:	learn: 0.3267010	total: 687ms	remaining: 885ms
437:	learn: 0.3266051	total: 689ms	remaining: 884ms
438:	learn: 

603:	learn: 0.3057837	total: 991ms	remaining: 650ms
604:	learn: 0.3056706	total: 993ms	remaining: 648ms
605:	learn: 0.3054845	total: 994ms	remaining: 646ms
606:	learn: 0.3051444	total: 996ms	remaining: 645ms
607:	learn: 0.3049924	total: 997ms	remaining: 643ms
608:	learn: 0.3046819	total: 999ms	remaining: 641ms
609:	learn: 0.3046209	total: 1s	remaining: 640ms
610:	learn: 0.3044342	total: 1s	remaining: 638ms
611:	learn: 0.3043408	total: 1s	remaining: 636ms
612:	learn: 0.3041825	total: 1s	remaining: 634ms
613:	learn: 0.3041014	total: 1.01s	remaining: 633ms
614:	learn: 0.3040476	total: 1.01s	remaining: 631ms
615:	learn: 0.3038884	total: 1.01s	remaining: 629ms
616:	learn: 0.3038671	total: 1.01s	remaining: 627ms
617:	learn: 0.3037660	total: 1.01s	remaining: 626ms
618:	learn: 0.3037357	total: 1.01s	remaining: 624ms
619:	learn: 0.3036407	total: 1.01s	remaining: 623ms
620:	learn: 0.3035905	total: 1.02s	remaining: 622ms
621:	learn: 0.3035026	total: 1.02s	remaining: 620ms
622:	learn: 0.3034567	to

829:	learn: 0.2820445	total: 1.33s	remaining: 273ms
830:	learn: 0.2819723	total: 1.34s	remaining: 272ms
831:	learn: 0.2819638	total: 1.34s	remaining: 270ms
832:	learn: 0.2818759	total: 1.34s	remaining: 269ms
833:	learn: 0.2816792	total: 1.34s	remaining: 267ms
834:	learn: 0.2815497	total: 1.34s	remaining: 265ms
835:	learn: 0.2815084	total: 1.34s	remaining: 264ms
836:	learn: 0.2813526	total: 1.35s	remaining: 262ms
837:	learn: 0.2811592	total: 1.35s	remaining: 261ms
838:	learn: 0.2811088	total: 1.35s	remaining: 259ms
839:	learn: 0.2809825	total: 1.35s	remaining: 257ms
840:	learn: 0.2809354	total: 1.35s	remaining: 256ms
841:	learn: 0.2807123	total: 1.35s	remaining: 254ms
842:	learn: 0.2806525	total: 1.36s	remaining: 253ms
843:	learn: 0.2804295	total: 1.36s	remaining: 251ms
844:	learn: 0.2803014	total: 1.36s	remaining: 250ms
845:	learn: 0.2802192	total: 1.36s	remaining: 248ms
846:	learn: 0.2801725	total: 1.36s	remaining: 246ms
847:	learn: 0.2801107	total: 1.36s	remaining: 245ms
848:	learn: 

,Model,Accuracy,Bal Acc.,Time
1,Skl GBM,84.916201,82.595520,0.109326
2,XGBoost,81.564246,78.787879,0.096251
3,CatBoost,81.564246,79.057971,1.856516
4,Skl HistGBM,81.005587,78.603426,0.363985
5,LightGBM,81.005587,78.333333,0.092904
6,Decision Tree,79.888268,77.694335,0.020753
7,Random Forest,79.888268,77.424242,0.186407
8,AdaBoost,79.888268,77.964427,0.102760
9,Extra Trees,77.094972,74.341238,0.170987


# Exercise 6 (3pts):
Define a 10 Fold cross validation strategy with:
- With stratification
- shuffle=True
- random_state=0

And train all the models in a for loop.

Tip you can use **[cross_val_predict](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html)** for both training and predict with 

In [162]:
"""
skf = model_selection.StratifiedKFold(n_splits=10,random_state=0, shuffle=True)
    # CODE HERE

"""
skf = model_selection.StratifiedKFold(n_splits=10,random_state=0, shuffle=True)
results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

"""
for model_name, model in tree_classifiers.items():
    start_time = time.time()
        
    # TRAIN AND GET PREDICTIONS USING cross_val_predict() and x,y
    pred = # CODE HERE

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_test, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_test, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              
"""
for model_name, model in tree_classifiers.items():
    start_time = time.time()
        
    # TRAIN AND GET PREDICTIONS USING cross_val_predict() and x,y
    pred = model_selection.cross_val_predict(model, X, y, cv=skf)

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)


results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

C:\Users\User\anaconda3\envs\AI_MachineLearning\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\User\anaconda3\envs\AI_MachineLearning\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:25:30] WARNING: ..\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:25:30] WARNING: ..\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\User\anaconda3\envs\AI_MachineLearning\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:25:30] WARNING: ..\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\User\anaconda3\envs\AI_MachineLearning\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\User\anaconda3\envs\AI_MachineLearning\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

266:	learn: 0.3611934	total: 1.84s	remaining: 5.05s
267:	learn: 0.3609781	total: 1.85s	remaining: 5.06s
268:	learn: 0.3608277	total: 1.85s	remaining: 5.04s
269:	learn: 0.3606029	total: 1.86s	remaining: 5.04s
270:	learn: 0.3603692	total: 1.87s	remaining: 5.02s
271:	learn: 0.3600678	total: 1.87s	remaining: 5.01s
272:	learn: 0.3600217	total: 1.88s	remaining: 5s
273:	learn: 0.3596989	total: 1.88s	remaining: 4.98s
274:	learn: 0.3593990	total: 1.89s	remaining: 4.97s
275:	learn: 0.3592807	total: 1.89s	remaining: 4.95s
276:	learn: 0.3590748	total: 1.89s	remaining: 4.94s
277:	learn: 0.3590158	total: 1.9s	remaining: 4.93s
278:	learn: 0.3587283	total: 1.91s	remaining: 4.93s
279:	learn: 0.3585353	total: 1.91s	remaining: 4.92s
280:	learn: 0.3584614	total: 1.92s	remaining: 4.91s
281:	learn: 0.3583659	total: 1.93s	remaining: 4.91s
282:	learn: 0.3581946	total: 1.93s	remaining: 4.89s
283:	learn: 0.3579816	total: 1.94s	remaining: 4.88s
284:	learn: 0.3578483	total: 1.94s	remaining: 4.87s
285:	learn: 0.35

473:	learn: 0.3319328	total: 2.68s	remaining: 2.98s
474:	learn: 0.3319200	total: 2.69s	remaining: 2.97s
475:	learn: 0.3319006	total: 2.69s	remaining: 2.96s
476:	learn: 0.3317143	total: 2.7s	remaining: 2.96s
477:	learn: 0.3315846	total: 2.71s	remaining: 2.96s
478:	learn: 0.3314605	total: 2.71s	remaining: 2.95s
479:	learn: 0.3313204	total: 2.71s	remaining: 2.94s
480:	learn: 0.3312187	total: 2.72s	remaining: 2.94s
481:	learn: 0.3311526	total: 2.73s	remaining: 2.93s
482:	learn: 0.3309951	total: 2.73s	remaining: 2.92s
483:	learn: 0.3309705	total: 2.73s	remaining: 2.92s
484:	learn: 0.3308075	total: 2.74s	remaining: 2.91s
485:	learn: 0.3306495	total: 2.74s	remaining: 2.9s
486:	learn: 0.3304647	total: 2.74s	remaining: 2.89s
487:	learn: 0.3303626	total: 2.75s	remaining: 2.88s
488:	learn: 0.3302794	total: 2.75s	remaining: 2.87s
489:	learn: 0.3301822	total: 2.75s	remaining: 2.87s
490:	learn: 0.3300875	total: 2.76s	remaining: 2.86s
491:	learn: 0.3300157	total: 2.76s	remaining: 2.85s
492:	learn: 0.

636:	learn: 0.3133649	total: 3.19s	remaining: 1.81s
637:	learn: 0.3132177	total: 3.19s	remaining: 1.81s
638:	learn: 0.3131612	total: 3.19s	remaining: 1.8s
639:	learn: 0.3130614	total: 3.19s	remaining: 1.79s
640:	learn: 0.3128894	total: 3.19s	remaining: 1.79s
641:	learn: 0.3128192	total: 3.19s	remaining: 1.78s
642:	learn: 0.3127257	total: 3.2s	remaining: 1.77s
643:	learn: 0.3126012	total: 3.2s	remaining: 1.77s
644:	learn: 0.3125823	total: 3.2s	remaining: 1.76s
645:	learn: 0.3124264	total: 3.2s	remaining: 1.75s
646:	learn: 0.3123792	total: 3.2s	remaining: 1.75s
647:	learn: 0.3123191	total: 3.21s	remaining: 1.74s
648:	learn: 0.3122386	total: 3.21s	remaining: 1.74s
649:	learn: 0.3122285	total: 3.21s	remaining: 1.73s
650:	learn: 0.3122193	total: 3.21s	remaining: 1.72s
651:	learn: 0.3121480	total: 3.21s	remaining: 1.71s
652:	learn: 0.3120698	total: 3.21s	remaining: 1.71s
653:	learn: 0.3119152	total: 3.22s	remaining: 1.7s
654:	learn: 0.3118835	total: 3.22s	remaining: 1.69s
655:	learn: 0.31177

858:	learn: 0.2904586	total: 3.87s	remaining: 634ms
859:	learn: 0.2903973	total: 3.87s	remaining: 630ms
860:	learn: 0.2902045	total: 3.87s	remaining: 625ms
861:	learn: 0.2901727	total: 3.88s	remaining: 621ms
862:	learn: 0.2900547	total: 3.88s	remaining: 616ms
863:	learn: 0.2898991	total: 3.88s	remaining: 611ms
864:	learn: 0.2898236	total: 3.88s	remaining: 606ms
865:	learn: 0.2897885	total: 3.88s	remaining: 601ms
866:	learn: 0.2896683	total: 3.88s	remaining: 596ms
867:	learn: 0.2895918	total: 3.89s	remaining: 591ms
868:	learn: 0.2895119	total: 3.89s	remaining: 586ms
869:	learn: 0.2894341	total: 3.89s	remaining: 581ms
870:	learn: 0.2893682	total: 3.89s	remaining: 576ms
871:	learn: 0.2893068	total: 3.89s	remaining: 572ms
872:	learn: 0.2890517	total: 3.89s	remaining: 567ms
873:	learn: 0.2889217	total: 3.9s	remaining: 562ms
874:	learn: 0.2888193	total: 3.9s	remaining: 557ms
875:	learn: 0.2887622	total: 3.9s	remaining: 552ms
876:	learn: 0.2885701	total: 3.9s	remaining: 547ms
877:	learn: 0.28

,Model,Accuracy,Bal Acc.,Time
1,Skl GBM,83.277217,81.027706,1.281351
2,CatBoost,83.052750,80.955805,24.637126
3,LightGBM,82.603816,80.977375,0.994852
4,Skl HistGBM,82.491582,80.831176,6.215572
5,XGBoost,81.593715,80.047455,2.169385
6,Random Forest,81.144781,79.848528,2.011051
7,AdaBoost,81.144781,79.903653,1.094027
8,Extra Trees,80.695847,79.208609,1.656155
9,Decision Tree,79.349046,78.060589,0.273813


# Exercise 7.1
Train with all data the best model

In [130]:
X

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
PassengerId,,,,,,,,
1,3,male,22.0,1,0,7.2500,S,Mr
2,1,female,38.0,1,0,71.2833,C,Mrs
3,3,female,26.0,0,0,7.9250,S,Miss
4,1,female,35.0,1,0,53.1000,S,Mrs
5,3,male,35.0,0,0,8.0500,S,Mr
...,...,...,...,...,...,...,...,...
887,2,male,27.0,0,0,13.0000,S,Rev
888,1,female,19.0,0,0,30.0000,S,Miss
889,3,female,NaN,1,2,23.4500,S,Miss


In [163]:
# best_model = tree_classifiers["SELECT MY BEST MODEL HERE"]
best_model = tree_classifiers[results_ord['Model'].iloc[0]]
# Fit best model with all data
best_model.fit(X_train,y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_data',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['Pclass', 'Age', 'SibSp',
                                                   'Parch', 'Fare']),
                                                 ('cat_data',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('ordinal',
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  unknown_value=-1))]),
                                                  ['Sex', 'Embarked',
                                                   'Title'])])),
                ('Skl GBM', GradientBoostingClassifier())])

# Exercise 7.2 (2pts)
With your best model, generate the predicitions for test data (x_test)

In [164]:
# test_pred = # Get the predictions for x_test
test_pred = best_model.predict(X_test)

# Exercise 7.3

Submit to kaggle.

- You can use the kaggle command line app. Check https://github.com/Kaggle/kaggle-api

In [165]:
sub = pd.DataFrame(test_pred, index=X_test.index, columns=["Survived"])
sub.head()

,Survived
PassengerId,
154,0
753,0
611,0
201,0
311,1


In [166]:
sub.to_csv("sub.csv")

In [118]:
!kaggle competitions submit -c titanic -f sub.csv -m "My submission message"

'kaggle' is not recognized as an internal or external command,
operable program or batch file.
